In [1]:
!pip install pennylane

Defaulting to user installation because normal site-packages is not writeable
  Using cached PennyLane-0.38.0-py3-none-any.whl (1.9 MB)
  Using cached autograd-1.7.0-py3-none-any.whl (52 kB)
  Using cached rustworkx-0.15.1-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
  Using cached autoray-0.6.12-py3-none-any.whl (50 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached networkx-3.3-py3-none-any.whl (1.7 MB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Using cached PennyLane_Lightning-0.38.0-cp310-cp310-manylinux_2_28_x86_64.whl (15.3 MB)
  Using cached cachetools-5.5.0-py3-none-any.whl (9.5 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.0
    Uninstalling numpy-2.1.0:
      Successfully uninstalled numpy-2.1.0


In [2]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt